In [1]:
import pandas as pd
import numpy as np
import glob

In [56]:
#Getting all the CSV, obtained from SPARQL queries.
allFiles = glob.glob("Data/*.csv")

In [57]:
display(allFiles)

['Data\\Geographe.csv',
 'Data\\Historien.csv',
 'Data\\Journaliste.csv',
 'Data\\Politicien.csv']

In [76]:
def Extract_function(allFiles):
    
    #Initiating the DataFrame
    Table = pd.DataFrame(columns=['issue', 'date', 'function', 'name'])
    K = 0
    while (K < int(len(allFiles))):
        #Creating a DataFrame and filling it with the excel's data
        df = pd.read_csv(allFiles[K],)
        Table = Table.append(df)
        K+=1
      
    #Cleaning the table
    for i in Table.columns:
        #Removing errors with accents
        Table[i] = Table[i].str.replace('Ã©' , 'é')
        
    
    #Extracting issue date
    Table['issue'] = Table['issue'] + Table.date.str[:10]
    Table['date'] = Table.date.str[:4]
    Table['Pulse'] = Table['name'] + ' était ' + Table['function'] + ' en ' + Table['date'] + ' #' + Table['function'] + ' #' + Table['issue']
    Table = Table.sort_values('date')

    return Table

In [77]:
display(Extract_function(allFiles))

,issue,date,function,name,Pulse
5263,GDL1807-07-14,1807,historien,M. de Muller,M. de Muller était historien en 1807 #historie...
4728,GDL1810-02-23,1810,historien de la Suisse,M. Jean Mûller,M. Jean Mûller était historien de la Suisse en...
4251,GDL1825-08-23,1825,historien,M. de Sismbndi,M. de Sismbndi était historien en 1825 #histor...
1270,JDG1826-07-06,1826,historien de la Suisse,M.H. Zschokke,M.H. Zschokke était historien de la Suisse en ...
3032,JDG1828-06-19,1828,historien,Robert GIutz-Blotzheim,Robert GIutz-Blotzheim était historien en 1828...
653,JDG1830-05-13,1830,historien,La Cretelle,La Cretelle était historien en 1830 #historien...
70,JDG1840-01-18,1840,géographe,Adrien Balbi,Adrien Balbi était géographe en 1840 #géograph...
937,JDG1840-04-23,1840,grand historien,Jean de Muller,Jean de Muller était grand historien en 1840 #...
4859,JDG1842-09-13,1842,journaliste,Elisée Lecomte,Elisée Lecomte était journaliste en 1842 #jour...
3629,JDG1843-01-17,1843,journaliste,Elisée Lenomte,Elisée Lenomte était journaliste en 1843 #jour...
